#### Day 24 - A
Get the logic gate output and combine all z values into a decimal number.

In [1]:
#Import Libraries and settings

settings = {
    "day": 24,
    "test_data": 0
}

In [2]:
def f_and(x, y):
    return x and y

def f_or(x, y):
    return x or y

def f_xor(x, y):
    return x ^ y

In [3]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    #Process inputs into dictionary
    circuit = {
        "in":{},
        "gate":{},
        "ready":[],
        "inactive":{"in":{}, "out":{}}
    }

    #Boolean to seperate the first half of the input from the second.
    initial  = True

    for line in lines:
        #If the break between the first and second second is found
        if line == "":
            initial = False
        #Process initial value for x 
        elif initial:
            wire = line.split(": ")
            wid = wire[0]
            
            wvalue = bool(int(wire[1]))
            circuit["in"][wid] = wvalue
        else:
            #Parse line
            gate_eq = line.split(" -> ")
            gate_in = gate_eq[0].split(" ")
            wires_in = [gate_in[0], gate_in[2]]
            gate_op = gate_in[1]
            gate_out = gate_eq[1]

            #Get operator function
            if gate_op == "AND":
                func = f_and
            elif gate_op == "OR":
                func = f_or
            elif gate_op == "XOR":
                func = f_xor

            #Record gate data
            circuit["gate"][gate_out] = {
                "in": wires_in[:],
                "op": func
                }
            
            #Record gate dependencies
            for wire in wires_in:
                if wire in circuit["inactive"]["in"].keys():
                    circuit["inactive"]["in"][wire].append(gate_out)
                else:
                    circuit["inactive"]["in"][wire] = [gate_out]

            circuit["inactive"]["out"][gate_out] = wires_in

    #Process the existing state to determine inactive and ready gates
    for wire in circuit["in"].keys():
        for connection in circuit["inactive"]["in"][wire]:
            circuit["inactive"]["out"][connection].remove(wire)

            if circuit["inactive"]["out"][connection] == []:
                del circuit["inactive"]["out"][connection]
                circuit["ready"].append(connection)

        del circuit["inactive"]["in"][wire]


    return circuit

circuit = load_input(settings)

In [4]:
def apply_gate_result(gate_id, val):
    global circuit

    #Add gate to inputs
    circuit["in"][gate_id] = val

    #Update inactive tracker
    if gate_id in circuit["inactive"]["in"].keys():
        for connection in circuit["inactive"]["in"][gate_id]:
                circuit["inactive"]["out"][connection].remove(gate_id)

                #Check if the output is expecting any more inputs
                if circuit["inactive"]["out"][connection] == []:
                    #Mark the output as ready to process
                    del circuit["inactive"]["out"][connection]
                    circuit["ready"].append(connection)

        #Remove processed gate from the inactive input list
        del circuit["inactive"]["in"][gate_id]

#Process a given gate id
def process_gate(gate_id):
    global circuit

    #Load gate details
    gate = circuit["gate"][gate_id]
    #Calculate result
    res = gate["op"](circuit["in"][gate["in"][0]], circuit["in"][gate["in"][1]])

    #Update circuit with the result
    apply_gate_result(gate_id, res)

    #Debug Line
    #print(gate_id, "=>", circuit["in"][gate["in"][0]], str(gate["op"]).split(" ")[1].split("_")[1].upper(), circuit["in"][gate["in"][1]], "=", circuit["in"][gate_id])

In [5]:
#Process ready circuits
while circuit["ready"]:
    next_gate = circuit["ready"].pop(0)
    process_gate(next_gate)
    

In [6]:
#Calculate the output
z_ids = [z for z in circuit["in"].keys() if z[0] == "z"]
z_ids.sort(reverse=True)
val = 0
for z in z_ids:
    #print(z, int(circuit["in"][z]))
    val = (val << 1) | int(circuit["in"][z])

print(val)

36902370467952
